In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.cross_validation import train_test_split
# Building a english to french translator

/Users/deveshmaheshwari/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [65]:
# Get the data
names=pd.read_csv('baby-names.csv')

In [66]:
names.name=names.name.apply(lambda x: x.lower())

In [67]:
names=pd.DataFrame({'name':names.name.unique()})

In [181]:
names.head()

,name,target
0,\tjohn,john\n
1,\twilliam,william\n
2,\tjames,james\n
3,\tcharles,charles\n
4,\tgeorge,george\n


In [182]:
lenght_list=[]
for l in names.name:
    lenght_list.append(len(l))
max_len = np.max(lenght_list)

In [68]:
# Append '\t' to input to delay target by 1 timstamp
names['name']=names.name.apply(lambda x:'\t'+x)

In [69]:
# Append \n to notify end of word
names['target']=names.name.apply(lambda x:x[1:len(x)]+'\n')

In [72]:
# Get the vocab dict
all_chars=set()
for name in names.name:
    for c in name:
        if c not in all_chars:
            all_chars.add(c)
all_chars.add('\n')

# max length of a name is 11
char_to_ix = { ch:i for i,ch in enumerate(sorted(all_chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(all_chars)) }

In [78]:
input_data = np.zeros((len(names.name), max_len, 28),dtype='float32')
output_data = np.zeros((len(names.name), max_len, 28),dtype='float32')

In [79]:
# Generate input and output data
for i, x in enumerate(names.name):
    for t, ch in enumerate(x):
        input_data[i, t, char_to_ix[ch]] = 1.
for i, x in enumerate(names.target):
    for t, ch in enumerate(x):
        output_data[i,t, char_to_ix[ch]] = 1.

In [184]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.callbacks import LambdaCallback
from keras.layers import TimeDistributed
from keras.optimizers import RMSprop
from keras.utils import plot_model

In [185]:
model = Sequential()
model.add(LSTM(50, input_shape=(max_len, len(all_chars)), return_sequences=True))
model.add(TimeDistributed(Dense(len(all_chars))))
model.add(TimeDistributed(Activation('softmax')))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [186]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 12, 50)            15800     
_________________________________________________________________
time_distributed_33 (TimeDis (None, 12, 28)            1428      
_________________________________________________________________
time_distributed_34 (TimeDis (None, 12, 28)            0         
Total params: 17,228
Trainable params: 17,228
Non-trainable params: 0
_________________________________________________________________


In [187]:
# Sampling function to generate new names
def onend(epoch, logs):
    if epoch%2==0 and epoch !=0:
        print('----- Generating text after Epoch: %d' % epoch)
        for i in range(0,10):
            stop=False
            ch='\t'
            counter=1
            target_seq = np.zeros((1, max_len, 28))
            target_seq[0, 0, char_to_ix[ch]] = 1.
            while stop == False and counter < 10:
                #sample the data
                probs = model.predict_proba(target_seq, verbose=0)[:,counter-1,:]
                c= np.random.choice(sorted(list(all_chars)), replace =False,p=probs.reshape(28))
                #c=ix_to_char[np.argmax(probs.reshape(28))]
                if c=='\n':
                    stop=True
                else:
                    ch=ch+c
                    target_seq[0,counter , char_to_ix[c]] = 1.
                    counter=counter+1
            print(ch)


In [189]:
# fit the model
print_callback = LambdaCallback(on_epoch_end=onend)
model.fit(input_data, output_data, batch_size=32,epochs=3, callbacks=[print_callback])

Epoch 1/3
6782/6782 [==============================] - 8s - loss: 1.1711     
Epoch 2/3
6782/6782 [==============================] - 8s - loss: 1.1493     
Epoch 3/3
6752/6782 [============================>.] - ETA: 0s - loss: 1.1317----- Generating text after Epoch: 2
	andi
	rubie
	carmand
	paile
	franceron
	venis
	juline
	darine
	judie
	phon
6782/6782 [==============================] - 8s - loss: 1.1317     
